## CityHack 2023 - Hackathon outcome prediction model by Bharat, Sahrish, Lek Shyuen

In [46]:
import re
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import hstack
import numpy as np

In [47]:
df=pd.read_csv("data.csv") 

In [48]:
df=df.dropna()

## data processing

In [49]:
def process_win(text):
    if ((text.find("[]")>=0) or text.find("{}")>=0):
        return 0
    return 1

def process_desc(text):
    return text.replace("\n","").strip()

def process_med(text):
    if (text=="[]"):
        return 0
    elif (text.count(",")==0):
        return 1
    return (text.count(",")+1)

def process_desclen(text):
    return len(text.strip().split(" "))

def process_title_punc(text):
    if isinstance(text,str):
        #return text.translate(str.maketrans('', '', string.punctuation))
        return re.sub(r'[^\w\s]', ' ', text)
    return str(text)

def process_author(text):
    if (text=="[]"):
        return 0
    elif (text.count(",")==0):
        return 1
    return (text.count(",")+1)

def remove_html(text):
    soup = BeautifulSoup(text)
    return soup.get_text()

def remove_urls(text):
    pattern = re.compile(r'https?://(www\.)?(\w+)(\.\w+)(/\w*)?')
    text = re.sub(pattern, "", text)
    return text

def remove_mentions(text):
    pattern = re.compile(r"@\w+")
    text = re.sub(pattern, "", text)
    return text

def remove_unicode_chars(text):
    text = text.encode("ascii", "ignore").decode()
    return text

def remove_extra_spaces(text):
    text = re.sub(' +', ' ', text).strip()
    return text

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

df['win1'] = df.win.apply(func = process_win)
df['desc'] = df.description.apply(func = remove_html)
df['med'] = df.media.apply(func = process_med)
df['desc_len'] = df.desc.apply(func = process_desclen)
df['title_rem'] = df.title.apply(func = process_title_punc)
df['author_c'] = df.authors.apply(func = process_author)
df['raw_desc'] = df.raw_desc.apply(func=remove_html)

df['raw_desc'] = df.raw_desc.str.lower()
df['description'] = df.description.str.lower()
df['tagline'] = df.tagline.str.lower()

/Users/lekshyuen/Documents/anaconda/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [50]:
df['desc'] = df.desc.apply(func=process_desc)
df['desc'] = df.desc.apply(func=remove_urls)

In [51]:
df.head(100)

,url,title,tagline,raw_desc,description,media,links,submitted,authors,win,win1,desc,med,desc_len,title_rem,author_c
0,https://devpost.com/software/proximity-wemo-au...,proximity-wemo-automator,restful service that talks to your wemo device...,\ntodo\n,\n todo \n,[],['https://github.com/ithai89/proximity-wemo-au...,[],['ivanthai'],{},0,TODO,0,1,proximity wemo automator,1
1,https://devpost.com/software/doggy-bell-tweete...,doggy-bell-tweeter,raspberry-pi project armed with a large paw fr...,\ntodo\ninspiration\nhow it works\nchallenges ...,\n todo \n inspiration \n how it works \n chal...,[],['https://github.com/ithai89/doggy-bell-tweeter'],[],['ivanthai'],{},0,TODO Inspiration How it works Challenges I ...,0,27,doggy bell tweeter,1
2,https://devpost.com/software/ringent,Ringent,ecommerce,\nhttp://ringent.com\nmaking an ecommerce site.\n,\n http://ringent.com \n making an ecommerce s...,['https://challengepost-s3-challengepost.netdn...,['http://ringent.com'],[],['chrism2chrism'],{},0,Making an ecommerce site.,2,6,Ringent,1
3,https://devpost.com/software/moveahead,MoveAhead,move ahead with the moverio! intelligent and h...,\noverview: a voice based gps navigation mover...,\n overview: a voice based gps navigation move...,['https://challengepost-s3-challengepost.netdn...,[],[],['rmj'],{},0,Overview: A voice based GPS navigation moverio...,1,64,MoveAhead,1
4,https://devpost.com/software/c-eye-or-c-i-y0q8a,C-Eye (or C-I),x-ray goggles,\nc-arm (a x-ray generator and monitoring syst...,\n c-arm (a x-ray generator and monitoring sys...,['https://challengepost-s3-challengepost.netdn...,[],[],['doctor26'],{},0,C-arm (a x-ray generator and monitoring system...,1,266,C Eye or C I,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,https://devpost.com/software/tamu-connect,TAMU Connect,"enter destination, get the bus route to reach ...",\ninspiration\nwhy does everyone use google ma...,\n inspiration \n why does everyone use google...,[],['https://diversityhackathon.wixsite.com/chall...,['https://tamu-hackathon.devpost.com/'],"['cveera1994', 'aniketunderveil', 'sillasampat...",{'https://tamu-hackathon.devpost.com/': []},0,Inspiration Why does everyone use google maps...,0,742,TAMU Connect,3
114,https://devpost.com/software/blossom-6m4gw2,Blossom,an app to personalize and streamline caring fo...,\nblossom\ntools used\napp\n\napache cordova\n...,\n blossom \n tools used \n app \n \n apache c...,['https://challengepost-s3-challengepost.netdn...,['https://github.com/astherath/Blossom'],['https://shellhacks-2019.devpost.com/'],"['astherath', 'emailofrafa']",{'https://shellhacks-2019.devpost.com/': []},0,Blossom Tools used App Apache cordova Mob...,12,41,Blossom,2
115,https://devpost.com/software/ffff-ic9rw0,ffff,fff,\ninspiration\nwhat it doesdddd\nhow i built i...,\n inspiration \n what it doesdddd \n how i bu...,[],[],[],['barasafa'],{},0,Inspiration What it doesdddd How I built it ...,0,30,ffff,1
116,https://devpost.com/software/safersection-dvqkxe,SaferSection,utilizing eye tracker data to develop safety m...,\nsafersection is optimizing intersection desi...,\n safersection is optimizing intersection des...,['https://www.youtube.com/embed/Ul8YndgQ0y0?en...,[],['https://ese450f17.devpost.com/'],"['KaylinRaby', 'OliviaWind', 'arslanm', 'Rache...",{'https://ese450f17.devpost.com/': []},0,Safersection is optimizing intersection design...,1,169,SaferSection,4


In [52]:
df1=df.drop(["url","title","description","media","links","authors","win","raw_desc","submitted"],axis=1)

In [53]:
df1.head(100)

,tagline,win1,desc,med,desc_len,title_rem,author_c
0,restful service that talks to your wemo device...,0,TODO,0,1,proximity wemo automator,1
1,raspberry-pi project armed with a large paw fr...,0,TODO Inspiration How it works Challenges I ...,0,27,doggy bell tweeter,1
2,ecommerce,0,Making an ecommerce site.,2,6,Ringent,1
3,move ahead with the moverio! intelligent and h...,0,Overview: A voice based GPS navigation moverio...,1,64,MoveAhead,1
4,x-ray goggles,0,C-arm (a x-ray generator and monitoring system...,1,266,C Eye or C I,1
...,...,...,...,...,...,...,...
112,"enter destination, get the bus route to reach ...",0,Inspiration Why does everyone use google maps...,0,742,TAMU Connect,3
114,an app to personalize and streamline caring fo...,0,Blossom Tools used App Apache cordova Mob...,12,41,Blossom,2
115,fff,0,Inspiration What it doesdddd How I built it ...,0,30,ffff,1
116,utilizing eye tracker data to develop safety m...,0,Safersection is optimizing intersection design...,1,169,SaferSection,4


In [54]:
df1["tagline"]=df1["tagline"].astype(str)
df1["desc"]=df1["desc"].astype(str)

In [55]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = [lemmatizer.lemmatize(word) for word in text.split()]
    text = ' '.join(words)
    return text

df1["tagline"]=df1["tagline"].apply(func=lemmatize_text)
df1["desc"]=df1["desc"].apply(func=lemmatize_text)
df1["title_rem"]=df1["title_rem"].apply(func=lemmatize_text)

In [56]:
df1.head()

,tagline,win1,desc,med,desc_len,title_rem,author_c
0,restful service that talk to your wemo device ...,0,TODO,0,1,proximity wemo automator,1
1,raspberry-pi project armed with a large paw fr...,0,TODO Inspiration How it work Challenges I ran ...,0,27,doggy bell tweeter,1
2,ecommerce,0,Making an ecommerce site.,2,6,Ringent,1
3,move ahead with the moverio! intelligent and h...,0,Overview: A voice based GPS navigation moverio...,1,64,MoveAhead,1
4,x-ray goggles,0,C-arm (a x-ray generator and monitoring system...,1,266,C Eye or C I,1


In [57]:
df2=df1.iloc[:3000]

In [83]:
tfidf = TfidfVectorizer(binary=True, analyzer='word',
            ngram_range=(1, 2), use_idf=True,smooth_idf=1,sublinear_tf=1)

X = tfidf.fit_transform(df2['tagline']).toarray()
temp1=tfidf.fit_transform(df2['desc']).toarray()
temp2=tfidf.fit_transform(df2['title_rem']).toarray()

X = hstack((X , np.array(df2['desc_len'])[:,None]))  
X = hstack((X , np.array(df2['author_c'])[:,None]))  
X = hstack((X , np.array(df2['med'])[:,None]))   
X = np.hstack((X,temp1))
X = np.hstack((X,temp2)) 
y=df2['win1']

tfidfX_train, tfidfX_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=42)
print (tfidfX_train.shape)
print (tfidfX_test.shape)
tfidfX_test

(2010, 324496)
(990, 324496)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [84]:
from imblearn.over_sampling import RandomOverSampler
RandomOverSampler = RandomOverSampler(random_state = 42)
tfidfX_train, y_train = RandomOverSampler.fit_resample(tfidfX_train, y_train)
print (tfidfX_train.shape)

(3276, 324496)


# Train model

In [85]:
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression()
clf1.fit(tfidfX_train, y_train)

predictions = clf1.predict_proba(tfidfX_test)
print("Test acc:", clf1.score(tfidfX_test, y_test))

/Users/lekshyuen/Documents/anaconda/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test acc: 0.8101010101010101


In [ ]:
"""
clf = GridSearchCV(LogisticRegression(), param_grid = parameters,scoring='accuracy',cv=10)
clf.fit(X_train,y_train)
print("Tuned Hyperparameters :", clf.best_params_)
print("Accuracy :",clf.best_score_)
"""

## Load model, predict probability

In [156]:
import pickle
#pickle.dump(clf1, open('model.pkl', 'wb'))
lg_model = pickle.load(open('model.pkl', 'rb'))

In [157]:
data = [{'tagline': "Automate your home using internet of things", 'desc': "This project revolves the idea of automating your home using a MSP430 along thje various sensors custom for the module", 
         'title': "home automation system","media_num":3,"author_num":1},
        {'tagline': "A web-based user friendly realtime inline debugger for JavaScript", 'desc': "An awesome disruptive advanced cross-platform scalable debugger for JavaScript", 'title': "InstaJS",
         "media_num":2,"author_num":4},
        {'tagline': "Save space. Save time. Shopping made easy.", 'desc': "We bring to you an innovative website where you can not only shop for the same goods that are available in the stores.", 'title': "NoCart",
         "media_num":4,"author_num":3}
       ]

df = pd.DataFrame(data)
  
df

,tagline,desc,title,media_num,author_num
0,Automate your home using internet of things,This project revolves the idea of automating y...,home automation system,3,1
1,A web-based user friendly realtime inline debu...,An awesome disruptive advanced cross-platform ...,InstaJS,2,4
2,Save space. Save time. Shopping made easy.,We bring to you an innovative website where yo...,NoCart,4,3


In [158]:
def process_win(text):
    if ((text.find("[]")>=0) or text.find("{}")>=0):
        return 0
    return 1

def process_desc(text):
    return text.replace("\n","").strip()

def process_med(text):
    if (text=="[]"):
        return 0
    elif (text.count(",")==0):
        return 1
    return (text.count(",")+1)

def process_desclen(text):
    return len(text.strip().split(" "))

def process_title_punc(text):
    if isinstance(text,str):
        #return text.translate(str.maketrans('', '', string.punctuation))
        return re.sub(r'[^\w\s]', ' ', text)
    return str(text)

def process_author(text):
    if (text=="[]"):
        return 0
    elif (text.count(",")==0):
        return 1
    return (text.count(",")+1)

def remove_html(text):
    soup = BeautifulSoup(text)
    return soup.get_text()

def remove_urls(text):
    pattern = re.compile(r'https?://(www\.)?(\w+)(\.\w+)(/\w*)?')
    text = re.sub(pattern, "", text)
    return text

def remove_mentions(text):
    pattern = re.compile(r"@\w+")
    text = re.sub(pattern, "", text)
    return text

def remove_unicode_chars(text):
    text = text.encode("ascii", "ignore").decode()
    return text

def remove_extra_spaces(text):
    text = re.sub(' +', ' ', text).strip()
    return text

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

df['desc'] = df.desc.apply(func = remove_html)
df['desc_len'] = df.desc.apply(func = process_desclen)
df['title'] = df.title.apply(func = process_title_punc)
df['tagline'] = df.tagline.str.lower()
df['desc'] = df.desc.apply(func=process_desc)
df['desc'] = df.desc.apply(func=remove_urls)

In [159]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = [lemmatizer.lemmatize(word) for word in text.split()]
    text = ' '.join(words)
    return text

df["tagline"]=df["tagline"].apply(func=lemmatize_text)
df["desc"]=df["desc"].apply(func=lemmatize_text)
df["title"]=df["title"].apply(func=lemmatize_text)

In [160]:
df.head()

,tagline,desc,title,media_num,author_num,desc_len
0,automate your home using internet of thing,This project revolves the idea of automating y...,home automation system,3,1,20
1,a web-based user friendly realtime inline debu...,An awesome disruptive advanced cross-platform ...,InstaJS,2,4,9
2,save space. save time. shopping made easy.,We bring to you an innovative website where yo...,NoCart,4,3,23


In [161]:
tfidf = TfidfVectorizer(binary=True, analyzer='word',
            ngram_range=(1, 2), use_idf=True,smooth_idf=1,sublinear_tf=1)

X = tfidf.fit_transform(df['tagline']).toarray()
temp1=tfidf.fit_transform(df['desc']).toarray()
temp2=tfidf.fit_transform(df['title']).toarray()

X = hstack((X , np.array(df['desc_len'])[:,None]))  
X = hstack((X , np.array(df['author_num'])[:,None]))  
X = hstack((X , np.array(df['media_num'])[:,None]))   
X = np.hstack((X,temp1))
X = np.hstack((X,temp2))

X=np.pad(X, ((0,0),(0,324496-X.shape[1])), mode='constant', constant_values=0)

predictions = lg_model.predict_proba(X)

df100 = pd.DataFrame()

for i in range(len(predictions)):
    data = [{'probability': predictions[i][0]}]
    df100=df100.append(data,ignore_index=True)


result = pd.concat([df,df100],axis=1)

result

/var/folders/86/4rbln91x06gbz0jyrncf0nzr0000gn/T/ipykernel_74796/1629942700.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df100=df100.append(data,ignore_index=True)
/var/folders/86/4rbln91x06gbz0jyrncf0nzr0000gn/T/ipykernel_74796/1629942700.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df100=df100.append(data,ignore_index=True)
/var/folders/86/4rbln91x06gbz0jyrncf0nzr0000gn/T/ipykernel_74796/1629942700.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df100=df100.append(data,ignore_index=True)


,tagline,desc,title,media_num,author_num,desc_len,probability
0,automate your home using internet of thing,This project revolves the idea of automating y...,home automation system,3,1,20,0.295154
1,a web-based user friendly realtime inline debu...,An awesome disruptive advanced cross-platform ...,InstaJS,2,4,9,0.705827
2,save space. save time. shopping made easy.,We bring to you an innovative website where yo...,NoCart,4,3,23,0.251286
